# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 2 2022 1:39PM,243293,10,0085921556,ISDIN Corporation,Released
1,Aug 2 2022 1:39PM,243293,10,0085921555,ISDIN Corporation,Released
2,Aug 2 2022 1:39PM,243293,10,0085921560,ISDIN Corporation,Released
3,Aug 2 2022 1:39PM,243293,10,0085921561,ISDIN Corporation,Released
4,Aug 2 2022 1:39PM,243293,10,0085921570,ISDIN Corporation,Released
5,Aug 2 2022 1:35PM,243292,10,Enova-10402,Emerginnova,Released
6,Aug 2 2022 1:35PM,243292,10,Enova-10403,Emerginnova,Released
7,Aug 2 2022 1:33PM,243291,10,SNAP59820,"Snap Medical Industries, LLC",Released
8,Aug 2 2022 1:33PM,243291,10,SNAP8793,"Snap Medical Industries, LLC",Released
9,Aug 2 2022 1:22PM,243289,10,ACI-8308395-BO,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
50,243288,Released,1
51,243289,Released,3
52,243291,Released,2
53,243292,Released,2
54,243293,Released,5


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243288,NaN,NaN,1.0
243289,NaN,NaN,3.0
243291,NaN,NaN,2.0
243292,NaN,NaN,2.0
243293,NaN,NaN,5.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243170,41.0,4.0,0.0
243173,0.0,9.0,7.0
243174,7.0,7.0,23.0
243204,4.0,15.0,0.0
243208,0.0,7.0,12.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243170,41,4,0
243173,0,9,7
243174,7,7,23
243204,4,15,0
243208,0,7,12


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243170,41,4,0
1,243173,0,9,7
2,243174,7,7,23
3,243204,4,15,0
4,243208,0,7,12


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243170,41,4,
1,243173,,9,7
2,243174,7,7,23
3,243204,4,15,
4,243208,,7,12


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 2 2022 1:39PM,243293,10,ISDIN Corporation
5,Aug 2 2022 1:35PM,243292,10,Emerginnova
7,Aug 2 2022 1:33PM,243291,10,"Snap Medical Industries, LLC"
9,Aug 2 2022 1:22PM,243289,10,"ACI Healthcare USA, Inc."
12,Aug 2 2022 1:10PM,243286,20,"Exact-Rx, Inc."
20,Aug 2 2022 1:08PM,243288,10,Yusen – 3D Matrix
21,Aug 2 2022 12:57PM,243287,10,ISDIN Corporation
35,Aug 2 2022 12:50PM,243208,20,"Exact-Rx, Inc."
54,Aug 2 2022 12:38PM,243278,16,American International Chemical
55,Aug 2 2022 12:34PM,243283,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 2 2022 1:39PM,243293,10,ISDIN Corporation,,,5
1,Aug 2 2022 1:35PM,243292,10,Emerginnova,,,2
2,Aug 2 2022 1:33PM,243291,10,"Snap Medical Industries, LLC",,,2
3,Aug 2 2022 1:22PM,243289,10,"ACI Healthcare USA, Inc.",,,3
4,Aug 2 2022 1:10PM,243286,20,"Exact-Rx, Inc.",,,8
5,Aug 2 2022 1:08PM,243288,10,Yusen – 3D Matrix,,,1
6,Aug 2 2022 12:57PM,243287,10,ISDIN Corporation,,,14
7,Aug 2 2022 12:50PM,243208,20,"Exact-Rx, Inc.",,7,12
8,Aug 2 2022 12:38PM,243278,16,American International Chemical,,1,
9,Aug 2 2022 12:34PM,243283,10,ISDIN Corporation,,,9


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 1:39PM,243293,10,ISDIN Corporation,5,,
1,Aug 2 2022 1:35PM,243292,10,Emerginnova,2,,
2,Aug 2 2022 1:33PM,243291,10,"Snap Medical Industries, LLC",2,,
3,Aug 2 2022 1:22PM,243289,10,"ACI Healthcare USA, Inc.",3,,
4,Aug 2 2022 1:10PM,243286,20,"Exact-Rx, Inc.",8,,
5,Aug 2 2022 1:08PM,243288,10,Yusen – 3D Matrix,1,,
6,Aug 2 2022 12:57PM,243287,10,ISDIN Corporation,14,,
7,Aug 2 2022 12:50PM,243208,20,"Exact-Rx, Inc.",12,7,
8,Aug 2 2022 12:38PM,243278,16,American International Chemical,,1,
9,Aug 2 2022 12:34PM,243283,10,ISDIN Corporation,9,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 1:39PM,243293,10,ISDIN Corporation,5,,
1,Aug 2 2022 1:35PM,243292,10,Emerginnova,2,,
2,Aug 2 2022 1:33PM,243291,10,"Snap Medical Industries, LLC",2,,
3,Aug 2 2022 1:22PM,243289,10,"ACI Healthcare USA, Inc.",3,,
4,Aug 2 2022 1:10PM,243286,20,"Exact-Rx, Inc.",8,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 1:39PM,243293,10,ISDIN Corporation,5.0,NaN,NaN
1,Aug 2 2022 1:35PM,243292,10,Emerginnova,2.0,NaN,NaN
2,Aug 2 2022 1:33PM,243291,10,"Snap Medical Industries, LLC",2.0,NaN,NaN
3,Aug 2 2022 1:22PM,243289,10,"ACI Healthcare USA, Inc.",3.0,NaN,NaN
4,Aug 2 2022 1:10PM,243286,20,"Exact-Rx, Inc.",8.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 1:39PM,243293,10,ISDIN Corporation,5.0,0.0,0.0
1,Aug 2 2022 1:35PM,243292,10,Emerginnova,2.0,0.0,0.0
2,Aug 2 2022 1:33PM,243291,10,"Snap Medical Industries, LLC",2.0,0.0,0.0
3,Aug 2 2022 1:22PM,243289,10,"ACI Healthcare USA, Inc.",3.0,0.0,0.0
4,Aug 2 2022 1:10PM,243286,20,"Exact-Rx, Inc.",8.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4135648,116.0,0.0,0.0
15,972861,35.0,17.0,21.0
16,243278,0.0,1.0,0.0
18,243232,1.0,0.0,0.0
19,1945909,6.0,18.0,4.0
20,1945833,56.0,31.0,55.0
21,729737,1.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4135648,116.0,0.0,0.0
1,15,972861,35.0,17.0,21.0
2,16,243278,0.0,1.0,0.0
3,18,243232,1.0,0.0,0.0
4,19,1945909,6.0,18.0,4.0
5,20,1945833,56.0,31.0,55.0
6,21,729737,1.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,116.0,0.0,0.0
1,15,35.0,17.0,21.0
2,16,0.0,1.0,0.0
3,18,1.0,0.0,0.0
4,19,6.0,18.0,4.0
5,20,56.0,31.0,55.0
6,21,1.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,116.0
1,15,Released,35.0
2,16,Released,0.0
3,18,Released,1.0
4,19,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21
Status,,,,,,,
Completed,0.0,21.0,0.0,0.0,4.0,55.0,0.0
Executing,0.0,17.0,1.0,0.0,18.0,31.0,2.0
Released,116.0,35.0,0.0,1.0,6.0,56.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21
0,Completed,0.0,21.0,0.0,0.0,4.0,55.0,0.0
1,Executing,0.0,17.0,1.0,0.0,18.0,31.0,2.0
2,Released,116.0,35.0,0.0,1.0,6.0,56.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21
0,Completed,0.0,21.0,0.0,0.0,4.0,55.0,0.0
1,Executing,0.0,17.0,1.0,0.0,18.0,31.0,2.0
2,Released,116.0,35.0,0.0,1.0,6.0,56.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()